<a href="https://colab.research.google.com/github/sristiraj/Cordova/blob/master/LegalData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [5]:
!wget -q https://downloads.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop2.tgz

In [6]:
!tar -xf spark-3.3.0-bin-hadoop2.tgz

In [7]:
!pip install -q findspark

In [8]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop2"

In [9]:
import findspark
findspark.init()

In [11]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName('test').config('spark.ui.port','4050').getOrCreate()

In [13]:
df = spark.read.csv("/content/lu45945bhceej.csv",header="true")

In [14]:
df.show()

+--------------+------------+------------+----------------------+--------------------+------------+
|DocumentNumber|DocumentDate|DocumentType|RefersToDocumentNumber|RefersToDocumentYear|     Remarks|
+--------------+------------+------------+----------------------+--------------------+------------+
|            27|  16/08/2021|           C|                  null|                null|        null|
|             1|  15/02/2021|           T|                     8|                2020|2020 0008-00|
|            67|  09/10/2020|           A|                  null|                null|        null|
|           157|  09/10/2020|           A|                  null|                null|        null|
|           189|  09/10/2020|           J|                  null|                null|        null|
|           250|  09/10/2020|           R|                  2016|                  98|        null|
|            87|  07/08/2020|           R|                  1992|                  97|        null|


In [15]:
df.createOrReplaceTempView("legaldata")

In [54]:
spark.sql('select DocumentNumber, DocumentDate from legaldata group by DocumentNumber, DocumentDate having count(*)>1').show()

+--------------+
|DocumentNumber|
+--------------+
|            15|
|            69|
|           133|
|           250|
|           139|
|            52|
|           100|
|            78|
|            89|
|            60|
|            23|
|            40|
|            82|
|           117|
|           180|
|            97|
|            88|
|             1|
|           237|
|            37|
+--------------+
only showing top 20 rows



In [36]:
spark.sql("select max(cast(DocumentNumber as int)) from legaldata").show()

+--------------------------------+
|max(CAST(DocumentNumber AS INT))|
+--------------------------------+
|                           30112|
+--------------------------------+



In [49]:
spark.sql("select doc.DocumentNumber,doc.DocumentNumber DocumentNumber,doc.DocumentType DocumentType, \
          doc.DocumentDate, doc.RefersToDocumentNumber, doc.RefersToDocumentYear,\
          doc.Remarks\
          from legaldata doc left outer join legaldata refdoc on doc.RefersToDocumentNumber = refdoc.DocumentNumber\
          where 1=1 \
          and ((refdoc.DocumentType='T' and doc.DocumentType='J') or (refdoc.DocumentType='R')) \
          union\
          select doc.DocumentNumber,doc.DocumentNumber DocumentNumber,doc.DocumentType DocumentType, \
          doc.DocumentDate, doc.RefersToDocumentYear RefersToDocumentNumber, doc.RefersToDocumentNumber RefersToDocumentYear,\
          doc.Remarks\
          from legaldata doc left outer join legaldata refdoc on doc.RefersToDocumentYear = refdoc.DocumentNumber\
          where 1=1 \
          and ((refdoc.DocumentType='T' and doc.DocumentType='J') or (refdoc.DocumentType='R'))").show()

+--------------+--------------+------------+------------+----------------------+--------------------+-------+
|DocumentNumber|DocumentNumber|DocumentType|DocumentDate|RefersToDocumentNumber|RefersToDocumentYear|Remarks|
+--------------+--------------+------------+------------+----------------------+--------------------+-------+
|            40|            40|           R|  11/01/2011|                    69|                1987|   null|
|            88|            88|           R|  18/07/1988|                    88|                1987|   null|
|           108|           108|           T|  24/12/2010|                    52|                2006|   null|
|            87|            87|           R|  07/08/2020|                    97|                1992|   null|
|            53|            53|           T|  14/10/1994|                    37|                1987|   null|
+--------------+--------------+------------+------------+----------------------+--------------------+-------+



In [51]:
spark.sql("select * from legaldata where DocumentNumber='69'").show()

+--------------+------------+------------+----------------------+--------------------+-------+
|DocumentNumber|DocumentDate|DocumentType|RefersToDocumentNumber|RefersToDocumentYear|Remarks|
+--------------+------------+------------+----------------------+--------------------+-------+
|            69|  15/10/2018|           C|                  null|                null|   null|
|            69|  01/02/1993|           R|                  1987|                 227|   null|
|            69|  24/07/1987|           B|                  null|                null|   null|
+--------------+------------+------------+----------------------+--------------------+-------+

